In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install Qiskit
!{sys.executable} -m pip install qiskit_ibm_runtime
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pylatexenc
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install qiskit_aer

## Quantum Initialization and the Streaming Operator

This notebook is based on following publications:
```
Schalkers, Merel A., and Matthias Möller. "Efficient and fail-safe collisionless quantum Boltzmann method." arXiv preprint arXiv:2211.14269 (2022).
Budinski, Ljubomir. "Quantum algorithm for the advection–diffusion equation simulated with the lattice Boltzmann method." Quantum Information Processing 20.2 (2021): 57.
Wawrzyniak, David, et al. "A quantum algorithm for the lattice-Boltzmann method advection-diffusion equation." Computer Physics Communications 306 (2025): 109373.
```


In this script we will create a quantum circuit for initialization (or state preperation) of the quantum system.
This essentially upload our classical data to the quantum computer.

Furthermore, we will implement two different algorithms for the streaming step of the QLBM. The first one is based on the Quantum Fourier Transform (QFT) and second on a cascaded Multicontrolled CNOT gates.
We will compare the computational complexity streaming of both approaches.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit import *

In [ ]:
#Helper functions
def is_power_of_two(vector : np.ndarray) -> bool:
    n = len(vector)
    return (n > 0) and (n & (n - 1)) == 0

## Initial Variables

In this section, we define the variables required for the simulation. These variables determine the size and structure of the quantum circuit and the domain of the system.

1. **Number of Qubits**:
   The $\texttt{Number\_of\_qubits}$ determines the size of the quantum circuit and, consequently, the domain size. The relationship between the number of qubits and the domain size is given by:
   
   $
   \texttt{Number\_of\_cells} = 2^{\texttt{Number\_of\_qubits}}.
   $

   Alternatively, if the domain size is predefined (and must be a power of two), the number of qubits can be computed as:
   
   $
   \texttt{Number\_of\_qubits} = \log_2(\texttt{Number\_of\_cells}).
   $

2. **Velocity Set**:
   In this workshop, we focus on the one-dimensional D1Q3 velocity set of the lattice Boltzmann method (LBM). This velocity set involves three distinct populations whose time evolution is computed during the simulation.

3. **Quantity of Interest**:
   As we solve the advection-diffusion equation, our primary quantity of interest is the time evolution of a scalar field, specifically the density. The density represents a distribution over the quantum states and evolves according to physical principles modeled in the simulation.

These initial variables provide a foundation for setting up and analyzing quantum simulations in this workshop.


## Initial Variables

Here we define some variables required for the simulation.
Number_of_qubits determines the size of our Quantum Circuit. We can either define number of qubits we are using in our simulation and therefore the domain size by $Number\_of\_cells=2^{Number\_of\_qubits}$
Or vice versa, we define a domain, which has to be a power of two and we compute the qubit number by $Number\_of\_qubits=log_2(Number\_of\_cells)$.

We consider in this workshop only the one dimensional D1Q3 velocity set of the LBM. 
In this velocity set the time evolution of three distinct population is computed.

As we solve the advection diffusion equation, our only quantity of interest is the time evolution of a scalar field. In this case the density.

In [ ]:
#Define initial condition

Number_of_qubits = 5
Number_of_distributions = 3
Number_of_distribution_qubits = np.ceil(np.log2(Number_of_distributions))


Total_number_of_cells = 2**Number_of_qubits

mid_index = Total_number_of_cells//2
density = np.linspace(0, 1, Total_number_of_cells)

In [ ]:
plt.plot(density, marker="o", linestyle="None",label='initial')
plt.legend()
plt.xlabel("x")
plt.ylabel("Density")
plt.title("Initial Condition")
plt.grid(True)
plt.show()

To simulate the D1Q3 velocity set, we need to initialize the density field three times, corresponding to the three distinct populations in the lattice Boltzmann method (LBM). Each population represents a component of the scalar field's time evolution.

### Initialization Constraints
Quantum systems can only represent amplitudes in vectors of size $2^n$, where $n$ is the number of qubits. This constraint arises from the structure of quantum state vectors, which live in a Hilbert space of dimension $2^n$. Therefore, if the desired initialization vector does not have a size that is a power of two, it must be padded with zeros to match this requirement.

If the density field has fewer elements than $2^n$, zeros can be appended to the vector until its size becomes $2^n$.


In [ ]:
#V1
#The vector we initialize must be 2**N sized
density_vector = np.tile(density, 3)
#### CODE MISSING HERE
density_vector = np.concatenate((density_vector, np.zeros_like(density)))
####
print(is_power_of_two(density_vector))

#normalize the vector for initialization
#### CODE MISSING HERE
normalization = np.linalg.norm(density_vector)
density_vector_normalized = density_vector / normalization
####

Initialize the Quantum Circuit.
Use two different QuantumRegisters. One for the encoding of the domain and one for the number of distribution functions.
Using two QuantumRegisters will structure the circuit and make implementations of the algorithm easier.
The qubits of the Registers can be accssesd by qx[qubit_number], qf[qubit_number]. This makes applying a quantum gate easier.

## Initialize the Quantum Circuit

To structure the quantum circuit and simplify the implementation of algorithms, we use two separate $\texttt{QuantumRegister}$ objects:
- One register encodes the domain size.
- The other register represents the number of distribution functions.

### Advantages of Using Multiple Quantum Registers
Using multiple registers organizes the circuit logically, making it easier to access individual qubits and apply gates. Qubits in each register can be accessed using their respective names and indices, such as $\texttt{qx[qubit\_number]}$ for the domain register and $\texttt{qf[qubit\_number]}$ for the distribution register.

### Steps to Initialize
1. **Define Quantum Registers**:
   - The domain register is initialized with a number of qubits sufficient to encode the domain size.
   - The distribution register is initialized with enough qubits to represent the number of distributions.

   For example:
   
   $
   \texttt{qx = QuantumRegister(Number\_of\_qubits, 'qx')}
   $

   $
   \texttt{qf = QuantumRegister(Number\_of\_distribution\_qubits, 'qf')}
   $

2. **Create a Quantum Circuit**:
   Combine the quantum registers into a single circuit:
   $
   \texttt{qc = QuantumCircuit(qx, qf)}
   $

3. **Access Qubits**:
   Individual qubits can now be accessed by name and index, simplifying gate application:
   
   $
   \texttt{qc.h(qx[0])} \quad \text{(Apply a Hadamard gate to the first qubit in the domain register).}
   $

   $
   \texttt{qc.cx(qx[0], qf[0])} \quad \text{(Apply a CNOT gate between registers).}
   $

In [ ]:
#### CODE MISSING HERE
qx = QuantumRegister(Number_of_qubits,'qx')
qf = QuantumRegister(Number_of_distribution_qubits,'qf')
####
cr = ClassicalRegister(Number_of_qubits+Number_of_distribution_qubits,'c')
qc = QuantumCircuit(qx, qf, cr)

To initialize a quantum state with a normalized density vector, we use the $\texttt{initialize}$ method provided by the Qiskit SDK. This method prepares the quantum circuit to start in a specific state, defined by the given vector of amplitudes.

### Initialization Process
1. **Normalized Density Vector**:
   The input to the $\texttt{initialize}$ method must be a normalized vector. This means that the sum of the squared magnitudes of all amplitudes must equal 1:
   
   $
   \sum_{i} |a_i|^2 = 1,
   $

   where $a_i$ are the amplitudes of the quantum state.

2. **Method Usage**:
   The initialization is performed as follows:
   
   $
   \texttt{qc.initialize(density\_vector\_normalized, [qx, qf])}.
   $

   Here:
   - $\texttt{density\_vector\_normalized}$ is the normalized state vector.
   - $\texttt{[qx, qf]}$ specifies the qubits to which the initialization applies. In this case, it spans both the domain register $(\texttt{qx})$ and distribution register $(\texttt{qf})$.

3. **Important Notes**:
   - The size of the state vector must match $2^n$, where $n$ is the total number of qubits in \texttt{[qx, qf]}. If necessary, padding with zeros can be applied to meet this requirement.
   - The initialization process resets all specified qubits and prepares them in the desired quantum state.

### Example
For a circuit with two registers ($\texttt{qx}$ and $\texttt{qf}$), you can initialize a custom state as follows:

$
\texttt{qc.initialize}([1/\sqrt{2}, 1/\sqrt{2}, 0, 0], [qx, qf]).
$

This prepares the first two qubits in an equal superposition of $\lvert 00 \rangle$ and $\lvert 01 \rangle$.

In [ ]:
#### CODE MISSING HERE
qc.initialize(density_vector_normalized, [qx,qf])
####
#barrier is only for visualization in the circuit
qc.barrier()
qc.draw('mpl')

The amplitudes of a quantum state vector are, in general, complex numbers. When using the $\texttt{Statevector(qc)}$ method to retrieve the state vector of a quantum circuit, the output will include both real and imaginary components for each amplitude. 

In this case, since we have initialized the quantum state with purely real amplitudes, the imaginary part of each amplitude is zero. Therefore, we extract only the real part of the state vector to analyze and plot our initialized density field.

In [ ]:
# Statevector is in general complex
Psi = np.real(Statevector(qc))

In [ ]:
plt.plot(Psi[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label='initial')
plt.legend()
plt.xlabel("x")
plt.ylabel("Psi Amplitudes")
plt.title("Initialized in Psi")
plt.grid(True)
plt.show()

## Streaming Operator

The streaming operator in the Lattice Boltzmann Method (LBM) shifts the entries of the density vector by one position in the positive (right) or negative (left) direction. This operator is linear and exact, making it suitable for quantum implementation. 

### Implementation Details
1. **Positive Streaming**:
   To implement streaming in the positive direction, we apply a cascading sequence of multi-controlled X (MCX) gates followed by an X gate. These gates act on the qubits of the domain register ($\texttt{qx}$) while leaving the distribution register ($\texttt{qf}$) for now untouched.

2. **Controlled Gates**:
   - The MCX gates ensure that the shift operation is applied conditionally based on the states of control qubits.
   - The X gate flips the target qubit, completing the shift operation.

To apply a multi-controlled X (MCX) gate in Qiskit, use the $\texttt{mcx()}$ method, specifying the control qubits and the target qubit. For example:

$
\texttt{qc.mcx(qx[:2], qx[2])}
$

applies an MCX gate with the first two qubits ($\texttt{qx[:2]}$) as control qubits and the third qubit ($\texttt{qx[2]}$) as the target qubit.

In [ ]:
qx = QuantumRegister(Number_of_qubits,'qx')
qf = QuantumRegister(Number_of_distribution_qubits,'qf')
cr = ClassicalRegister(Number_of_qubits+Number_of_distribution_qubits,'c')
qc = QuantumCircuit(qx, qf, cr)
qc.initialize(density_vector_normalized, [qx,qf])
#barrier is only required for visualization
qc.barrier()

# Implement streaming circuit using multi controlled x-gates -> positive

#### CODE MISSING HERE
for i in range(len(qx)-1, 0, -1):  
    qc.mcx(qx[:i], qx[i])  
qc.x(qx[0])
#### 
qc.draw(output='mpl')

Check if streaming has been applied correctly.

In [ ]:
Psi_new = np.real(Statevector(qc))
plt.plot(Psi[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label='initial')
plt.plot(Psi_new[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label='shifted')
plt.legend()
plt.xlabel("x")
plt.ylabel("Psi Amplitudes")
plt.title("Initialized in Psi")
plt.grid(True)
plt.show()

## Streaming Operator in the Negative Direction

The streaming operator in the negative direction shifts the entries of the density vector by one position to the left. In quantum mechanics, every operator is reversible, meaning that the inverse of an operator can undo its effects. For the streaming operator, this property ensures:

$
U_{\text{STREAMING\_POS}} \cdot U_{\text{STREAMING\_NEG}} = I,
$
where $I$ is the identity operator.

### Negative Streaming Implementation
The streaming operator in the negative direction can be implemented as the inverse of the positive streaming operator:

$
U_{\text{STREAMING\_NEG}} = U_{\text{STREAMING\_POS}}^{-1}.
$

In the our case of the streaming operator, the inverse corresponds to reversing the sequence of gates applied during positive streaming.

In [ ]:
qx = QuantumRegister(Number_of_qubits,'qx')
qf = QuantumRegister(Number_of_distribution_qubits,'qf')
cr = ClassicalRegister(Number_of_qubits+Number_of_distribution_qubits,'c')
qc = QuantumCircuit(qx, qf, cr)
qc.initialize(density_vector_normalized, [qx,qf])
#barrier is only required for visualization
qc.barrier()

# Implement streaming circuit using multi controlled x-gates -> negative
#### CODE MISSING HERE
qc.x(qx[0])  
for i in range(1, len(qx)):
    qc.mcx(qx[:i], qx[i])
####
qc.draw(output='mpl')

Check if streaming has been applied correctly.

In [ ]:
Psi_new = np.real(Statevector(qc))
plt.plot(Psi[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label='initial')
plt.plot(Psi_new[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label='shifted')
plt.legend()
plt.xlabel("x")
plt.ylabel("Psi Amplitudes")
plt.title("Initialized in Psi")
plt.grid(True)
plt.show()

## Streaming Operator with Controlled Subspaces

In the Lattice Boltzmann Method (LBM), each population has its own independent streaming direction. To implement this in the quantum circuit, we apply the streaming operator only to specific subspaces of the state vector. This is achieved by using control operations on the qubits in the $\texttt{qf}$ register, which represent the distribution functions.

### Controlled Streaming
1. **Uncontrolled Streaming**:
   Applying a streaming operator directly to the $\texttt{qx}$ register without any control on the $\texttt{qf}$ register shifts the entire state vector:
   
   $
   U_{\text{STREAM}} \lvert q_f \rangle \lvert q_x \rangle.
   $

2. **Controlled Streaming**:
   To apply streaming to a specific subspace, we use control gates on the $\texttt{qf}$ register. For example:
   - To apply streaming to the subspace $\lvert 11 \rangle_f \lvert q_x \rangle$, we control the streaming operator on all qubits of the $\texttt{qf}$ register.
   - To apply streaming to $\lvert 01 \rangle_f \lvert q_x \rangle$, we first apply an $X$ gate to flip the second qubit of the $\texttt{qf}$ register, then apply the controlled streaming operator, and finally apply another $X$ gate to restore the original state.

### D1Q3 Velocity Set
For the D1Q3 velocity set in LBM:
- $f_0$: The resting distribution is not streamed.
- $f_1$: Streamed in the positive direction.
- $f_2$: Streamed in the negative direction.

This translates into the quantum circuit as follows:

$
\lvert 00 \rangle_f \lvert q_x \rangle : \text{No operation (resting distribution)},
$

$
U_{\text{positive\_streaming}} \lvert 01 \rangle_f \lvert q_x \rangle : \text{Streaming in positive direction},
$

$
U_{\text{negative\_streaming}} \lvert 10 \rangle_f \lvert q_x \rangle : \text{Streaming in negative direction}.
$

### Implementation
To implement this behavior:
1. Use control gates on the entire $\texttt{qf}$ register to target specific subspaces.
2. Apply $X$ gates as necessary to adjust controls for states like $\lvert 01 \rangle_f$ or $\lvert 10 \rangle_f$.
3. Combine these operations with $U_{\text{positive\_streaming}}$ and $U_{\text{negative\_streaming}}$ to achieve the desired streaming for each population.


In [ ]:
#only shift specific distributions

qx = QuantumRegister(Number_of_qubits,'qx')
qf = QuantumRegister(Number_of_distribution_qubits,'qf')
cr = ClassicalRegister(Number_of_qubits+Number_of_distribution_qubits,'c')
qc = QuantumCircuit(qx, qf, cr)
qc.initialize(density_vector_normalized, [qx,qf])
#barrier is only required for visualization
qc.barrier()

# Implement streaming circuit using multi controlled x-gates -> positive

#### CODE MISSING HERE
qc.x(qf[1])
####

for i in range(len(qx)-1, 0, -1):  
    qc.mcx(qx[:i]+qf[:], qx[i])  
qc.mcx(qf[:],qx[0])

#### CODE MISSING HERE
qc.x(qf[1])
####

qc.barrier()

# Implement streaming circuit using multi controlled x-gates -> negative

#### CODE MISSING HERE
qc.x(qf[0])
####

qc.mcx(qf[:],qx[0])
for i in range(1, len(qx)):  # Reverse the MCX sequence
    qc.mcx(qx[:i]+qf[:], qx[i])

#### CODE MISSING HERE
qc.x(qf[0])
####
qc.barrier()



qc.draw(output='mpl')

Check if streaming has been applied correctly.

In [ ]:
Psi_new = np.real(Statevector(qc))
plt.plot(Psi[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None", label="Initial")
plt.plot(Psi_new[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label="Shifted")
plt.legend()

plt.xlabel("x")
plt.ylabel("Psi Amplitudes")
plt.title("Psi")
plt.grid(True)
plt.show()

## Streaming using the Quantum Fourier Transform

For a unitary operator that performs an evolution of the state vector, the quantum circuit implementation is not unique. Different approaches can be used to implement a specific operator. Here, we implement the streaming operator using the Quantum Fourier Transform (QFT).

### Quantum Fourier Transform
The QFT acts on a state vector $\lvert x \rangle$ as follows:

$
\text{QFT} \lvert x \rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i \frac{xk}{N}} \lvert k \rangle,
$

where $N = 2^n$ is the dimension of the state vector for $n$ qubits. $n$ are the number of qubits in the $|qx\rangle$ register.
Taking a closer look at the exponential function in the Quantum Fourier Transform (QFT) basis, the transformed state is represented as:

$
e^{2\pi i \frac{xk}{N}},
$

where the state label $x$ from the computational basis now appears in the argument of the exponential function. This allows us to modify the state label by applying a phase gate.

### Phase Gate
The phase gate modifies the phase of a quantum state and is represented by the following matrix:

$
P(\theta) = 
\begin{bmatrix}
1 & 0 \\
0 & e^{i\theta}
\end{bmatrix}.
$

In this implementation, we use the phase gate to adjust the exponential term in the Fourier basis to correspond to a constant $+1$ addition in the state label.


### Implementation Steps
1. **Apply QFT**: Transform the state vector into the Fourier basis using the QFT.
2. **Apply Phase Rotations**: Add a constant $+1$ to the state by applying controlled phase rotations in the Fourier basis. The rotation angles for each qubit are given by:
   
   $
   \theta_k = 2^{x+1} \cdot \frac{\pi}{N},
   $
   
   where $x$ is the qubit index, and $n$ is the total number of qubits in the domain register ($\texttt{qx}$).
   For the computation of the negative streaming, so adding a $-1$ use the negative value of $\theta_k$.
3. **Inverse QFT**: Transform back to the computational basis by applying the inverse QFT.

### Explanation
In the Fourier basis, adding a constant $+1$ corresponds to applying phase shifts to the amplitudes of the state vector. When transforming back to the computational basis, these phase shifts result in a cyclic shift of the states, effectively adding $+1$ to each index.

In [ ]:
#Second Algorithm

from qiskit.circuit.library import QFT
from numpy import pi

## Quantum Fourier Transform and Controlled Phase Gates

To implement the streaming operator using the Quantum Fourier Transform (QFT), we follow these steps:

1. **Apply the Quantum Fourier Transform**:
   The QFT is applied to all qubits in the domain register ($\texttt{qx}$) using:
   
   $
   \texttt{qc.append(qft\_circuit, qx[:])}.
   $

   The QFT transforms the state vector into the Fourier basis, where the state labels are encoded in the arguments of exponential functions.

2. **Apply Controlled Phase Gates**:
   After transforming into the Fourier basis, we apply controlled phase gates to modify the state labels. The control qubits are taken from the distribution register ($\texttt{qf}$), and the phase gates are applied to each qubit in $\texttt{qx}$ with angles given by:
   
   $
   \theta_x = 2^{x+1} \cdot \frac{\pi}{N},
   $

   where $k$ is the index of the qubit in $\texttt{qx}$, and $n$ is the total number of qubits in $\texttt{qx}$.

   For negative streaming use 
   
   $
   \theta_x = -2^{x+1} \cdot \frac{\pi}{N}.
   $

   The multi-controlled phase gate is applied using:

   $
   \texttt{qc.mcp(qf[:], qx[ii])}.
   $

3. **Transform Back to Computational Basis**:
   After applying the phase gates, we use the inverse QFT to transform back to the computational basis. This completes the process of adding a constant $+1$ to the state labels.

   $
   \texttt{qc.append(qft\_inverse\_circuit, qx[:])}.
   $

4. **Why do we add +1?

   $
   \theta_x = -2^{x+1} \cdot \frac{\pi}{N}.
   $
   
   The phase gate adds the amplitude $e^{i2\pi 2^x/N}$, therefore if it is applied on a basis state $|k\rangle$ it is multiplied by $e^{i2\pi k/N}$ and therefore

   $
   \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i \frac{xk}{N}}e^{i2\pi \frac{k}{N}} \lvert k \rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i \frac{(x+1)k}{N}}
   $

In [ ]:
qx = QuantumRegister(Number_of_qubits,'qx')
qf = QuantumRegister(Number_of_distribution_qubits,'qf')
cr = ClassicalRegister(Number_of_qubits+Number_of_distribution_qubits,'c')
qc = QuantumCircuit(qx, qf, cr)
qc.initialize(density_vector_normalized, [qx,qf])

qft_circuit = QFT(Number_of_qubits, do_swaps=True, inverse=False, approximation_degree=0)
qft_inverse_circuit = QFT(Number_of_qubits, do_swaps=True, inverse=True, approximation_degree=0)

#Positive streaming
qc.append(qft_circuit, qx[:])
qc.x(qf[1])
for x in range(len(qx)):
    #### CODE MISSING HERE
    #compute the phase angle
    theta = 2**(x+1)*pi/(2**(len(qx)))
    #apply mcp 
    qc.mcp(theta,[qf[0], qf[1]],qx[x])
    #### 

qc.x(qf[1])


qc.barrier()
#Negative streaming
qc.x(qf[0])
for x in range(len(qx)):
    #### CODE MISSING HERE
    #compute the phase angle
    theta = 2**(x+1)*pi/(2**(len(qx)))
    #apply mcp
    qc.mcp(-theta,[qf[0], qf[1]],qx[x])
    ####

qc.x(qf[0])
qc.append(qft_inverse_circuit, qx[:])
qc.barrier()



qc.draw(output='mpl')

Check if streaming has been applied correctly.

In [ ]:
Psi_new = np.real(Statevector(qc))
plt.plot(Psi[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None", label="Initial")
plt.plot(Psi_new[:3*Total_number_of_cells]*normalization, marker="o", linestyle="None",label="Shifted")

plt.legend()

plt.xlabel("x")
plt.ylabel("Psi Amplitudes")
plt.title("Psi")
plt.grid(True)
plt.show()